# Spotify Music Recommendation System

In [1]:
import os
import numpy as np
import pandas as pd
import re 
import json 
import sys
import itertools
from pprint import pprint


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
import sys
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv(r'C:\Users\adity\OneDrive\Documents\GitHub\Music Recommender system\data.csv')
genre_data = pd.read_csv(r'C:\Users\adity\OneDrive\Documents\GitHub\Music Recommender system\data_by_genres.csv')
year_data = pd.read_csv(r'C:\Users\adity\OneDrive\Documents\GitHub\Music Recommender system\data_by_year.csv')

In [3]:
#genre_data.info()

In [4]:
#genre_data.head()

In [5]:
genre_data.values[60][1]

'alternative rock'

In [6]:
#year_data.head()

In [7]:
#data.info()

In [8]:
#data.head()

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re 

In [12]:
client_id = '4379e8dc929f4f3d8b296962fb549d56'
client_secret= '08a15f73be3742f18227aec6fb9e61b4'
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()
        
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='https://mysongrecommender/dashboard/')
sp = spotipy.Spotify(auth=token)
    
    

Enter the URL you were redirected to: https://mysongrecommender/dashboard/?code=AQCMILUK4gbeyDxxGHXDdrbgAVJxzvqKyYIl342pVo7fjGmIYTpuFjEh-TkRJthTLhE4sUwK1FTaxr2p7agXHRatV8k7Jm3k4-oIvonAA07a0Kx138CNbIySwJqVZoXyu5-u1xzoCo7G1GQkyPrM6vivMP5Wr41F-fGURl-3o1P6ip7gsrhhKg


In [14]:
def ari_to_features(ari):
#Audio features
    features = sp.audio_features(ari)[0]
    
    #Artist of the track, for genres and popularity
    artist = sp.track(ari)["artists"][0]["id"]
    artist_pop = sp.artist(artist)["popularity"]
    artist_genres = sp.artist(artist)["genres"]
    
    #Track popularity
    track_pop = sp.track(ari)["popularity"]
    
    #Add in extra features
    features["artist_pop"] = artist_pop
    if artist_genres:
        features["genres"] = " ".join([re.sub(' ','_',i) for i in artist_genres])
    else:
        features["genres"] = "unknown"
    features["track_pop"] = track_pop
    
    return features

if __name__ == "__main__":
    # Debug
    result = ari_to_features("1o0nAjgZwMDK9TI4TTUSNn")
    print(result)

{'danceability': 0.8, 'energy': 0.599, 'key': 0, 'loudness': -7.357, 'mode': 1, 'speechiness': 0.0344, 'acousticness': 0.00521, 'instrumentalness': 0.00547, 'liveness': 0.0587, 'valence': 0.908, 'tempo': 128.381, 'type': 'audio_features', 'id': '1o0nAjgZwMDK9TI4TTUSNn', 'uri': 'spotify:track:1o0nAjgZwMDK9TI4TTUSNn', 'track_href': 'https://api.spotify.com/v1/tracks/1o0nAjgZwMDK9TI4TTUSNn', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1o0nAjgZwMDK9TI4TTUSNn', 'duration_ms': 225666, 'time_signature': 4, 'artist_pop': 58, 'genres': 'ann_arbor_indie funk_rock instrumental_funk', 'track_pop': 53}


In [19]:
data_path = "C:/Users/adity/OneDrive/Documents/GitHub/Spotify-Music-Recommendation/data/First_1000.json"
raw_json = json.loads(open(data_path).read())

# Transform Data
playlists = raw_json["playlists"]
df = pd.json_normalize(playlists, record_path='tracks', meta=['name'])

# Output
df.to_csv("C:/Users/adity/OneDrive/Documents/GitHub/Spotify-Music-Recommendation/data/rawdata.csv")

In [30]:
rawdataPath = 'C:/Users/adity/OneDrive/Documents/GitHub/Spotify-Music-Recommendation/data/rawdata.csv'
raw_data = pd.read_csv(rawdataPath)
raw_data.head(2)

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks
1,1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks


In [31]:
len(raw_data)

67503

In [32]:
raw_data["track_uri"] = raw_data["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
raw_data["track_uri"]

0        0UaMYEvWZi0ZqiDOoHU3YI
1        6I9VzXrHxO9rA9A5euc8Ak
2        0WqIKmW4BTrj3eJFmnCKMv
3        1AWQoqb9bSvzTjaLralEkT
4        1lzr43nnXAijIGYnCT8M8H
                  ...          
67498    5uCax9HTNlzGybIStD3vDh
67499    0P1oO2gREMYUCoOkzYAyFu
67500    2oM4BuruDnEvk59IvIXCwn
67501    4Ri5TTUgjM96tbQZd5Ua7V
67502    5RVuBrXVLptAEbGJdSDzL5
Name: track_uri, Length: 67503, dtype: object

In [33]:
ari_to_features(df["track_uri"][0])

{'danceability': 0.904,
 'energy': 0.813,
 'key': 4,
 'loudness': -7.105,
 'mode': 0,
 'speechiness': 0.121,
 'acousticness': 0.0311,
 'instrumentalness': 0.00697,
 'liveness': 0.0471,
 'valence': 0.81,
 'tempo': 125.461,
 'type': 'audio_features',
 'id': '0UaMYEvWZi0ZqiDOoHU3YI',
 'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
 'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI',
 'duration_ms': 226864,
 'time_signature': 4,
 'artist_pop': 69,
 'genres': 'dance_pop hip_hop hip_pop pop pop_rap r&b rap urban_contemporary virginia_hip_hop',
 'track_pop': 67}

In [34]:
first_half = df["track_uri"].unique()[:10000]
second_half = df["track_uri"].unique()[10000:20000]
third_half = df["track_uri"].unique()[20000:]
dataLIST = [first_half,second_half,third_half]

In [ ]:
from tqdm import tqdm
featureLIST = []

for i in tqdm([uri for uri in dataLIST[0]]):
    try:
        featureLIST.append(ari_to_features(i))
    except:
        continue

 29%|██▉       | 2942/10000 [1:34:27<1128:43:25, 575.72s/it]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3Gvpsucd1JbG8QqYZFRKNK with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4Lb1Kt73wdW7iOF5jkG2wZ with Params: {} returned 401 due to The access token expired
 29%|██▉       | 2944/10000 [1:34:28<607:41:36, 310.05s/it] HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6E7mh5MHmrA3TR57Y3p1j1 with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2ExAV3NX1HvJNVIFOtr2rK with Params: {} returned 401 due to The access token expired
 29%|██▉       | 2946/10000 [1:34:28<366:10:41, 186.88s/it]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=65hSpz9HeIx21NZZJ8iyGl with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-f

 30%|██▉       | 2984/10000 [1:34:30<30:13,  3.87it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5ITsHwjDEpOWvzWfNp0yjk with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4UFxT3SIb314tvoAv8WPTi with Params: {} returned 401 due to The access token expired
 30%|██▉       | 2986/10000 [1:34:30<23:53,  4.89it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3LPZpcPVseTmaS8DRcvkXN with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3aKNGl01j8n9RZY7XHWPMz with Params: {} returned 401 due to The access token expired
 30%|██▉       | 2988/10000 [1:34:31<18:53,  6.19it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=08GujSR7BcjakZUzX0R0nl with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5Net

 30%|███       | 3031/10000 [1:34:33<06:17, 18.46it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=18vVJsD68uh386vlZWwO3d with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6L4PjBaEjTxI0sOBGq8OQg with Params: {} returned 401 due to The access token expired
 30%|███       | 3033/10000 [1:34:33<07:04, 16.42it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2l8sAa7La9rjejJAmHdPEB with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4XIdxJBiWXUaQiJY5hlSIx with Params: {} returned 401 due to The access token expired
 30%|███       | 3035/10000 [1:34:33<06:43, 17.25it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=05CrK6Q5VGtfPDtyQFJ4Kf with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=03gp

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=32JAv27m214fWmEGNNCZ8L with Params: {} returned 401 due to The access token expired
 31%|███       | 3078/10000 [1:34:36<07:17, 15.82it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2kBpsXKH2hCUDslEcvS0FK with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1uwY6Jh3XZguvRQ9vc7mXp with Params: {} returned 401 due to The access token expired
 31%|███       | 3080/10000 [1:34:36<09:05, 12.68it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5OiLJ8tjUPFiPX2gVM8fxJ with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=52ZiapOPlSsnOW38efJZeK with Params: {} returned 401 due to The access token expired
 31%|███       | 3082/10000 [1:34:36<09:33, 12.05it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0p8S

 31%|███       | 3123/10000 [1:34:39<06:08, 18.65it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3sNVsP50132BTNlImLx70i with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7gjm1sq5ALU5svBSi9g178 with Params: {} returned 401 due to The access token expired
 31%|███▏      | 3125/10000 [1:34:39<06:27, 17.76it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2SpEHTbUuebeLkgs9QB7Ue with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2LA5bXro30zZGXca3JAhNt with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3wfujdbamR3Z46F4xav7LM with Params: {} returned 401 due to The access token expired
 31%|███▏      | 3128/10000 [1:34:39<06:12, 18.46it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0ipL

...

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7Gs3otHnZDq514kFHf0nx7 with Params: {} returned 401 due to The access token expired
 94%|█████████▎| 9365/10000 [1:40:40<00:35, 17.64it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5trW3r6R5Eobr8Z35gXE6o with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7MTIDmToGs0I5Oue9V0CHl with Params: {} returned 401 due to The access token expired
 94%|█████████▎| 9367/10000 [1:40:40<00:35, 17.79it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6CdkAjZAHPYkV6C9wjMi0L with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5B0WISmCAb2WFrpFsVdK5E with Params: {} returned 401 due to The access token expired
 94%|█████████▎| 9369/10000 [1:40:40<00:35, 17.66it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3uYm

 94%|█████████▍| 9410/10000 [1:40:43<00:35, 16.48it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4GjRDgb77p27GQnbZpv1OJ with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2ACOWPLUe4A4KuQ5ioD2od with Params: {} returned 401 due to The access token expired
 94%|█████████▍| 9412/10000 [1:40:43<00:35, 16.55it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6AIte2Iej1QKlaofpjCzW1 with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0YywjDvFudcaHG74NuWISy with Params: {} returned 401 due to The access token expired
 94%|█████████▍| 9414/10000 [1:40:43<00:33, 17.37it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4x0wdP2Kux6GYj2bAWHyO9 with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6fTd

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6xT6POT2QEHBZN4mI6sb7c with Params: {} returned 401 due to The access token expired
 95%|█████████▍| 9457/10000 [1:40:46<00:31, 17.51it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5RePFldQaYKHcZ4sLYg88a with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2VBo6OXRntDFDCzeXYZq7D with Params: {} returned 401 due to The access token expired
 95%|█████████▍| 9459/10000 [1:40:46<00:30, 17.58it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1u1XTfuVFbc8RM2qZPMvfG with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2xl1L3AX0N7rLyPvfschwA with Params: {} returned 401 due to The access token expired
 95%|█████████▍| 9461/10000 [1:40:46<00:32, 16.67it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0DNQ

 95%|█████████▌| 9502/10000 [1:40:48<00:28, 17.27it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4aaG9fGe7d8fe7ZWJEb1f0 with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0FGTz3d60ejPOHzJi1lxvN with Params: {} returned 401 due to The access token expired
 95%|█████████▌| 9504/10000 [1:40:48<00:28, 17.70it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4r2a0P7EQ9vcoJ9U6GNARq with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7IQWs82nSWONuLdFJ08y7Y with Params: {} returned 401 due to The access token expired
 95%|█████████▌| 9506/10000 [1:40:48<00:27, 18.08it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=54ZUtj02w5gVqlpT0n95bM with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0bLS

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4bhoQCNYx5TjINvtSMiaAb with Params: {} returned 401 due to The access token expired
 95%|█████████▌| 9549/10000 [1:40:51<00:26, 17.15it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4uQ7wYsuL0DryknoDc11Hk with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=297kL99O1dAFpm3UmtYsfm with Params: {} returned 401 due to The access token expired
 96%|█████████▌| 9551/10000 [1:40:51<00:25, 17.49it/s]HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0za4yuB3UmITV2L9QMwlV0 with Params: {} returned 401 due to The access token expired
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5T4j3Uv33fQO7tsR2lMl4y with Params: {} returned 401 due to The access token expired
 96%|█████████▌| 9554/10000 [1:51:33<9:49:35, 79.32s/it] 

In [ ]:
#Exploratory Data Analysis

In [ ]:
corr_df=data.drop(["key","mode","year","explicit"],axis=1).corr(method="pearson")
plt.figure(figsize=(16,7))
heatmap=sns.heatmap(corr_df,annot=True,fmt=".1g", vmin=-1, vmax=1, center=0, cmap="rocket", linewidths=1, linecolor="Black")
heatmap.set_title("Correlation HeatMap")
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=90)

In [ ]:
sample_data=data.sample(int(0.01*len(data)))
plt.figure(figsize=(12,6))
sns.regplot(data=sample_data, y="popularity", x="acousticness",color="purple").set(title="Popularity vs Acousticness Correlation")

In [ ]:
plt.figure(figsize=(12,6))
sns.regplot(data=sample_data, y="popularity", x="energy",color="blue").set(title="Popularity vs Energy Correlation")

In [ ]:
##Music Analysis over a decade

In [ ]:
def get_decade(year):
    period_start = int(year) - (int(year)%10)
    decade = '{}s'.format(period_start)
    return decade

data['decade'] = data['year'].apply(get_decade)

sns.set(rc={'figure.figsize':(11 ,6)})
sns.countplot(data['decade'])

In [ ]:
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']
fig = px.line(year_data, x='year', y=sound_features)
fig.show()


In [ ]:
#Popular genres and their characteristics

In [ ]:
top_genres = genre_data.nlargest(15, 'popularity')

fig = px.bar(top_genres, x='genres', y=['valence', 'energy', 'danceability','liveness'], barmode='group')
fig.show()

In [ ]:
##Clustering Genres with Kmeans

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=10, init='k-means++'))])
X = genre_data.select_dtypes(np.number)
cluster_pipeline.fit(X)
genre_data['cluster'] = cluster_pipeline.predict(X)

In [ ]:
from sklearn.manifold import TSNE

tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=1))])
genre_embedding = tsne_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = genre_data['genres']
projection['cluster'] = genre_data['cluster']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()

In [ ]:
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans(n_clusters=20, 
                                   verbose=False, init='k-means++'))
                                 ], verbose=False)

X = data.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
data['cluster_label'] = song_cluster_labels

In [ ]:
from sklearn.decomposition import PCA

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = data['name']
projection['cluster'] = data['cluster_label']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()


### Using Spotify API to build recommendation system

In [ ]:
!pip install spotipy

In [ ]:
client_id = '4379e8dc929f4f3d8b296962fb549d56'
client_secret= '08a15f73be3742f18227aec6fb9e61b4'

In [ ]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='https://mysongrecommender/dashboard/')
sp = spotipy.Spotify(auth=token)


In [ ]:
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [ ]:
id_name

In [ ]:
def create_necessary_outputs(playlist_name,id_dic, df):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_name (str): name of the playlist you'd like to pull from the spotify API
        id_dic (dic): dictionary that maps playlist_name to playlist_id
        df (pandas dataframe): spotify datafram
        
    Returns: 
        playlist: all songs in the playlist THAT ARE AVAILABLE IN THE KAGGLE DATASET
    """
    
    #generate playlist dataframe
    playlist = pd.DataFrame()
    playlist_name = playlist_name

    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
        #print(i['track']['artists'][0]['name'])
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [ ]:
id_name

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_songs(df):
    """ 
    Visualize cover art of the songs in the inputted dataframe

    Parameters: 
        df (pandas dataframe): Playlist Dataframe
    """
    
    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [ ]:
playlist_Heart = create_necessary_outputs('Wilson’s Heart', id_name,tracks_data)
playlist_Heart

In [ ]:
visualize_songs(playlist_Heart)


In [ ]:
def generate_playlist_feature(tracks_data, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [ ]:
complete_feature_set_playlist_vector_heart, complete_feature_set_nonplaylist_heart = generate_playlist_feature(complete_feature_set, playlist_Heart, 1.09)
#complete_feature_set_playlist_vector_chill, complete_feature_set_nonplaylist_chill = generate_playlist_feature(complete_feature_set, playlist_chill, 1.09)
complete_feature_set_playlist_vector_heart.shape

In [ ]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [ ]:
edm_top40 = generate_playlist_recos(tracks_data, complete_feature_set_playlist_vector_heart, complete_feature_set_nonplaylist_heart)

In [ ]:
edm_top40

In [ ]:
#visualize_songs(edm_top40)

In [ ]:
def find_song(name, year):
    song_data = defaultdict()
    results = sp.search(q= 'track: {} year: {}'.format(name,year), limit=1)
    if results['tracks']['items'] == []:
        return None

    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

    song_data['name'] = [name]
    song_data['year'] = [year]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]

    for key, value in audio_features.items():
        song_data[key] = value

    return pd.DataFrame(song_data)

In [ ]:
from collections import defaultdict
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib

number_cols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']


def get_song_data(song, spotify_data):
    
    try:
        song_data = spotify_data[(spotify_data['name'] == song['name']) 
                                & (spotify_data['year'] == song['year'])].iloc[0]
        return song_data
    
    except IndexError:
        return find_song(song['name'], song['year'])
        

def get_mean_vector(song_list, spotify_data):
    
    song_vectors = []
    
    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        if song_data is None:
            print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
            continue
        song_vector = song_data[number_cols].values
        song_vectors.append(song_vector)  
    
    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)


def flatten_dict_list(dict_list):
    
    flattened_dict = defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key] = []
    
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
            
    return flattened_dict


def recommend_songs( song_list, spotify_data, n_songs=10):
    
    metadata_cols = ['name', 'release_date', 'artists']
    song_dict = flatten_dict_list(song_list)
    
    song_center = get_mean_vector(song_list, spotify_data)
    scaler = song_cluster_pipeline.steps[0][1]
    scaled_data = scaler.transform(spotify_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])
    
    rec_songs = spotify_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs[metadata_cols].to_dict(orient='records')

In [ ]:
recommend_songs([{'name': 'Come As You Are', 'release_date':1991},
                {'name': 'Smells Like Teen Spirit', 'release_date': 1991},
                {'name': 'Lithium', 'release_date': 1992},
                {'name': 'All Apologies', 'release_date': 1993},
                {'name': 'Stay Away', 'release_date': 1993}],  tracks_data)